<a href="https://colab.research.google.com/github/sumankanukollu/EVA_4/blob/master/suman_eva4_s4_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from IPython.core.debugger import set_trace

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
          nn.Conv2d(1, 16, 3),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Dropout(0.1))
        self.conv2 = nn.Sequential(
          nn.Conv2d(16, 32, 3),
          nn.ReLU(),
          nn.BatchNorm2d(32),
          nn.Dropout(0.1))
        
        self.conv3 = nn.Conv2d(32,10,1)

        self.pool1 = nn.MaxPool2d(2,2)

        self.conv4 = nn.Sequential(
          nn.Conv2d(10, 10, 3),
          nn.ReLU(),
          nn.BatchNorm2d(10),
          nn.Dropout(0.1))
        
        self.conv5 = nn.Sequential(
          nn.Conv2d(10, 16, 3),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Dropout(0.1))
        
        self.conv6 = nn.Sequential(
          nn.Conv2d(16, 16, 3),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Dropout(0.1))
        
        self.conv7 = nn.Sequential(
          nn.Conv2d(16, 16, 3),
          nn.ReLU(),
          nn.BatchNorm2d(16),
          nn.Dropout(0.1))
        
        self.conv8 = nn.Sequential(
          nn.Conv2d(16, 10, 4),
          nn.ReLU(),
          nn.BatchNorm2d(10),
          nn.Dropout(0.1))
        
        

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.pool1(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        
        x = x.view(-1, 10)
        return F.log_softmax(x)


In [3]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 26, 26]             160
              ReLU-2           [-1, 16, 26, 26]               0
       BatchNorm2d-3           [-1, 16, 26, 26]              32
           Dropout-4           [-1, 16, 26, 26]               0
            Conv2d-5           [-1, 32, 24, 24]           4,640
              ReLU-6           [-1, 32, 24, 24]               0
       BatchNorm2d-7           [-1, 32, 24, 24]              64
           Dropout-8           [-1, 32, 24, 24]               0
            Conv2d-9           [-1, 10, 24, 24]             330
        MaxPool2d-10           [-1, 10, 12, 12]               0
           Conv2d-11           [-1, 10, 10, 10]             910
             ReLU-12           [-1, 10, 10, 10]               0
      BatchNorm2d-13           [-1, 10, 10, 10]              20
          Dropout-14           [-1, 10,

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:

torch.manual_seed(1)
batch_size = 128
#batch_size = 25088

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        #set_trace()
        #print(data.size())
        #print(target.size())
        optimizer.zero_grad()
        output = model(data)
        
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    #set_trace()
    print('\n##### Epoch Number : %d ######'%epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0894, Accuracy: 9820/10000 (98%)


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0539, Accuracy: 9862/10000 (99%)


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0236, Accuracy: 9933/10000 (99%)


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0250, Accuracy: 9936/10000 (99%)


loss=0.2047276496887207 batch_id=468: 100%|██████████| 469/469 [00:12<00:00, 36.27it/s]



Test set: Average loss: 0.0227, Accuracy: 9940/10000 (99%)
